In [35]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import os

Defining batch size & random seed. 

In [36]:
dataset_dir = "gestures"
batch_size=1024
seed=123

Load all gestures from disk and label them accordingly

In [37]:
gestures = {
  "Computer": [],
  "Elephant": [],
  "Sorry": [],
  "No-Sign": [],
}

dataset_values = []
dataset_labels = []

for gesture in gestures:
  for filename in os.listdir(f"gestures/{gesture}"):
    with open(os.path.join(f"gestures/{gesture}", filename), 'r') as f:
        text = f.read()
        lines = text.splitlines()
        for line in lines:
          
          intValues = []
          for value in line.split():
            value = int(value)
            intValues.append(value)

          gestures[gesture].append(intValues)
        
          if gesture == "Computer":
            dataset_labels.append(np.array([1,0,0,0]))
          elif gesture == "Elephant":
            dataset_labels.append(np.array([0,1,0,0]))
          elif gesture == "Sorry":
            dataset_labels.append(np.array([0,0,1,0]))
          elif gesture == "No-Sign":
            dataset_labels.append(np.array([0,0,0,1]))
          else:
            print("You did not change the labeling of the dataset you blithering idiot")

  gestures[gesture] = np.array(gestures[gesture])
  dataset_values.append(np.array(gestures[gesture]))

dataset_values = np.array(dataset_values)
dataset_labels = np.array(dataset_labels)

